In [5]:
from numpy import *
import math
import copy
import cPickle as pickle

class ID3Tree(object):
    def __init__(self):
        self.tree = {}    #生成的树
        self.dataSet = [] #数据集
        self.labels = []  #标签集
    
    def loadDataSet(self, path, labels):
        recordlist = []
        fp = open(path, "rb") 
        content = fp.read()
        fp.close()
        rowlist = content.splitlines()     #按行转换为一维表
        
    def train(self):
        labels = copy.deepcopy(self.labels)
        self.tree = self.buildTree(self.dataSet, labels)
        
    def buildTree(self, dataSet, labels):
        cateList = [data[-1] for data in dataSet]  #抽取源数据集的决策标签列
        #程序终止条件 1：如果classList只有一种决策标签，停止划分，返回这个决策标签
        if cateList.count(cateList[0]) == len(cateList):
            return cateList[0]
        
        #程序终止条件 2：如果数据集的第一个决策标签只有一个，返回这个决策标签
        if len(dataSet[0]) == 1:
            return self.macCate(cateList)
        
        #算法核心
        
        bestFeat = self.getBestFeat(dataSet)  #返回数据集的最优特征轴
        bestFeatLabel = labels[bestFeat]
        tree = {bestFeatLabel:{}}
        del(labels[bestFeat])
        
        #抽取最优特征轴的列向量
        uniqueVals = set([data[bestFeat] for data in dataSet]) #去重
        for value in uniqueVals:  #决策树递归生长
            subLabels = labels[:] #将删除后的特征类别集建立子类别集
            
            #按最优特征列和值分隔数据集
            splitDataSet = self.splitDataSet(dataSet, bestFeat, value)
            subtree = self.buildTree(splitDataSet, subLabels) #构建子树
            tree[bestFeatLabel][value] = subtree
        return tree
                
    def maxCate(self, catelist): #计算出现次数最多的类别标签
        items = dict([(catelist.count(i), 1) for i in catelist])
        return items[max(items.keys())]
    
    def getBestFeat(self, dataSet): #计算最优特征
        #计算特征向量维，其中最后一列用于类别标签，因此要减去
        numFeatures = len(dataSet[0] - 1)
        baseEntropy = self.computeEntropy(dataSet)
        bestInfoGain = 0.0
        bestFeature = -1
        #外循环：遍历数据集各列，计算最优特征轴
        # i 为数据集索引：取值范围 0-（numFeatures - 1)
        for i in xrange(numFeatures):
            uniqueVals = set([data[i] for data in dataSet])
            newEntropy = 0.0
            for value in uniqueVals:
                #按选定列i和唯一值分隔数据集
                subDataSet = self.splitDataSet(dataSet, i, value)
                prob = len(subDataSet) / float(len(dataSet))
                newEntropy += prob * self.computeEntropy(subDataSet)
            infoGain = baseEntropy - newEntropy
             
            if (infoGain > bestInfoGain): #如果信息增益大于0
                bestInfoGain = infoGain   #如果当前信息增益替代之前的最优增益值
                bestFeature = i           #重置最优特征为当前列
        return bestFeature
    
    def computeEntropy(self, dataSet):    #计算香农熵
        datalen = float(len(dataSet))
        cateList = [data[-1] for data in dataSet] #从数据集中得到类别标签
        #得到类别为key，出现次数为value的字典
        
        items = dict([(i, cateList.count(i)) for i in cateList])
        infoEntropy = 0.0  #初始化香农熵
        
        for key in items:
            prob = float(items[key]) / datalen
            infoEntropy -= prob * math.log(prob, 2)
        return infoEntropy
    
    #dataSet：数据集 axis：特征轴 values：特征轴的取值
    def splitDataSet(self, dataSet, axis, value): #划分数据集
        rtnList = []
        for featVec in dataSet:
            if featVec[axis] == value:
                rFeatVec = featVec[:value]   #List操作：提取0-（axis-1)的元素
                rFeatVec.extend(featVec[axis+1:])   #list操作：将特征轴之后的元素相加
                rtnList.append(rFeatVec)
        return rtnList
        